In [1]:
import sklearn.datasets

In [3]:
from sklearn.datasets import fetch_openml

data = fetch_openml(name='credit-g')

In [4]:
cat_cols = ['checking_status','credit_history','purpose','savings_status','savings_status','personal_status','other_parties','employment',
           'property_magnitude','other_payment_plans','housing','existing_credits','job','class']

In [5]:
df = data.frame

In [6]:
import pandas as pd
import numpy as np
for col in cat_cols:
    df[col] = pd.factorize(df[col])[0]

In [7]:
x_cols = [col for col in df.columns if col not in ['class']]
x = df[x_cols]
y = df['class']

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from skopt.optimizer import Optimizer
from skopt.space import Integer,Real
import warnings
warnings.filterwarnings("ignore")
def myscore(y_pred,y):
    return roc_auc_score(y,y_pred)

def f(para,seed=0):

        
    leaves,lr,n_estimators,child,subsample,colsample = para[0], 0.1, para[1], para[2], para[3], para[4]
    
    x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.3, random_state=seed)
    model =        LGBMClassifier(
                num_leaves=leaves,
                learning_rate=lr,
                n_estimators=n_estimators,
                min_child_samples=child,
                subsample = subsample,
                colsample_bytree = colsample,

                random_state = seed,
                n_jobs = -1
            )
    model.fit(x_train,y_train,
              categorical_feature = cat_cols
             
             )
#     print(model.predict_proba(x_test)[:,1],y_test.values)
    score = myscore(model.predict_proba(x_test)[:,1],y_test.values)
    
    return score
    
space = [Integer(4,256),Integer(1,1000),Integer(1,100),Real(0.1,1),Real(0.1,1)]
def ask(history,base_estimator,KAPPA=2.23):
        
    optimizer =  Optimizer( space,base_estimator=base_estimator, n_initial_points=1,initial_point_generator='random',
                acq_optimizer="auto", random_state=0,acq_func_kwargs = {'kappa':KAPPA},
                              acq_func='EI'
                              )
    for i in range(len(history)):
            config, loss = history[i]
            optimizer.tell(config, -loss,
                          fit=(i == len(history) - 1)
                          )
    
    a = optimizer.ask()
    return a

In [10]:
from tqdm.notebook import tqdm

In [17]:
fin_score_dict = {
    'GP':[],
    'RF':[],
    'ET':[],
    'GBRT':[],
}
n_round = 100
for test_bo in  fin_score_dict:

    for allseed in range(10):
        history = [] 
        init_num = 10
        optimizer =  Optimizer( space,base_estimator='GP', n_initial_points=10,initial_point_generator='random',
                    acq_optimizer="auto", random_state=allseed,
                                  acq_func='EI',#acq_func_kwargs = {'kappa':5}
                                  )
        for i in range(init_num):
            tmp_ask = optimizer.ask()
            tmp_score = f(tmp_ask,allseed)
            history.append((tmp_ask,tmp_score))


        for i in tqdm(range(n_round)):
            tmp_ask = ask(history,base_estimator=test_bo)
            tmp_score = f(tmp_ask)
            history.append((tmp_ask,tmp_score))
        fin_score_dict[test_bo].append(np.max(tmp_score))

In [18]:
def get_bo(mydict,history):
    score = [h[1] for h in history]
    base = np.mean(score)
    base_std = np.std(score)
    prob = []
    bo = []
    for idx in mydict:
        if len(mydict[idx]) == 0:
            prob.append(0.5)
        else:
            prob.append(max(0.1,(np.max(mydict[idx] - base) / base_std)))
        bo.append(idx)
    prob = np.array(prob)
    prob /= np.sum(prob)
    prob_cum = np.cumsum(prob)
    
    x = np.random.random()
    for i in range(len(bo)):
        if x <= prob_cum[i]:
            return bo[i]
    

In [19]:
mixscore = []
n_round = 100
for allseed in range(10):
    history = [] 
    init_num = 10
    optimizer =  Optimizer( space,base_estimator='GP', n_initial_points=10,initial_point_generator='random',
                acq_optimizer="auto", random_state=allseed,
                              acq_func='EI',#acq_func_kwargs = {'kappa':5}
                              )
    mydict = {
        'GP':[],
        'RF':[],
        'ET':[],
        'GBRT':[],
    }

    for i in range(init_num):

        tmp_ask = optimizer.ask()
        tmp_score = f(tmp_ask,allseed)
        history.append((tmp_ask,tmp_score))

    
    for i in tqdm(range(n_round)):
        bo = get_bo(mydict,history)
        tmp_ask = ask(history,base_estimator=bo)
        tmp_score = f(tmp_ask)
        history.append((tmp_ask,tmp_score))
        mydict[bo].append(tmp_score)
    mixscore.append(np.max(tmp_score))